In [2]:
import numpy as np
import random
import copy

In [3]:
class TicTacToe:
    def __init__(self):
        self.board = np.zeros(9)
    
    def Move(self, player_id, spot):
        if self.LegalMove(spot):
            self.board[spot] = player_id
            if self.CheckWin():
                return 1
            elif self.GetLegalMoves() == []:
                return 0
            else:
                return -1
    
    def LegalMove(self, spot):
        if self.board[spot] == 1 or self.board[spot] == -1:
            return False
        else:
            return True
        
    def GetLegalMoves(self):
        legal_moves = []
        
        for i in range(9):
            if self.LegalMove(i):
                legal_moves.append(i)
                
        return legal_moves
        
        
        
    def CheckWin(self):
        board = self.board
        if board[0] == board[1] and board[0] == board[2] and board[0] != 0:
            return True
        elif board[3] == board[4] and board[3] == board[5] and board[3] != 0:
            return True
        elif board[6] == board[7] and board[6] == board[8] and board[6] != 0:
            return True
        elif board[0] == board[3] and board[0] == board[6] and board[0] != 0:
            return True
        elif board[1] == board[4] and board[1] == board[7] and board[1] != 0:
            return True
        elif board[2] == board[5] and board[2] == board[8] and board[2] != 0:
            return True
        elif board[0] == board[4] and board[0] == board[8] and board[0] != 0:
            return True
        elif board[2] == board[4] and board[2] == board[6] and board[2] != 0:
            return True
        else:
            return False 

In [49]:
class Node:
    def __init__(self, name, game, player_id, parent = None):
        self.parent = parent
        self.children = []
        self.player_id = player_id
        self.visits = 0
        self.value = 0
        self.name = name
        self.game = copy.deepcopy(game)
        self.legalMoves = self.game.GetLegalMoves()
        
    def __repr__(self):
        return "(" + str(self.name) + ", " + str(self.visits) + ", " + str(self.value) + ")"
    
    def FindChild(self, name):
        for i in self.children:
            if i.name == name:
                return i
    
    def MakeChild(self, name, game, player_id):
        self.children.append(Node(parent = self, name = name, game = game, player_id = player_id))
        
    def UCTScore(self, C = 1.414):
        return self.value/self.visits + C * np.sqrt(np.log(self.parent.visits)/self.visits)
    
    def BestChild(self, C = 1.414):
        childrenScores = [(x.UCTScore(C),x) for x in self.children]      
        childrenScores = sorted(childrenScores,  key=lambda x: x[0])
        return childrenScores[-1][1]

In [152]:
def MCTS(game, iterations, player_id):
    
    head = Node(name = None, game = game, player_id = player_id)
    node = head
    
    # Select and Create
    for i in range(iterations):
        
        if node.legalMoves == [] and node.children == []:
            break   
            
        while(node.legalMoves == [] and node.children != []):
            node = node.BestChild()
        
        gameCopy = copy.deepcopy(node.game) 
        move = random.choice(node.legalMoves)
        gameCopy.Move(node.player_id, move)
        node.MakeChild(name = move, game = gameCopy, player_id = node.player_id * -1)
        node.legalMoves.remove(move)
        node = node.FindChild(move)    
        
        if node.legalMoves == [] and node.children == []:
            if move == 1:
                node.value += 1
            while node.parent != None:
                node.visits += 1
                node = node.parent
            node.visits += 1
            break   
        
        
        # Simulate
        copyCopy = copy.deepcopy(gameCopy)
        
        score = 0
        while(True):
            
            
            a = copyCopy.Move(node.player_id, random.choice(copyCopy.GetLegalMoves()))
           
            if a == 1:
                score = 1
                break
            elif a == 0:
                score = -1
                break
            elif copyCopy.GetLegalMoves() == []:
                break

            b = copyCopy.Move(node.player_id * -1, random.choice(copyCopy.GetLegalMoves()))    
        
            if b == 1:
                score = -1
                break
            elif copyCopy.GetLegalMoves() == []:
                break
        
            
        # Backpropigate 
        
        while node.parent != None:
            
            node.value += score
            node.visits += 1
            node = node.parent
            
        node.visits += 1
    
    visitedChildren = sorted(head.children, key = lambda x: x.visits)
    return visitedChildren[-1].name

    

In [143]:
win_percent = []


iterations = [(x+1)*200 for x in range(1)]

for i in iterations:
    win = 0
    tie = 0
    for num_games in range(100):
        board = TicTacToe()
        while(True):
            move = MCTS(board, i, 1)
            a = board.Move(1, move)
            if a == 1:
                win += 1
                break
            elif a == 0:
                tie += 1
                break
            
            
            move = random.choice(board.GetLegalMoves())
            b = board.Move(-1, move)
            if b == 0 or b == 1:
                break
                
    win_percent.append([win, tie, (100-win-tie)])
    print(win_percent)
    

[[87, 2, 11]]


In [ ]:
import matplotlib.pyplot as plt
plt.plot(win_percent)
plt.show()

1

In [383]:
a= [(4,2), (2,3)]
a.sort()
a

[(2, 3), (4, 2)]

In [83]:
a.head.legalMoves

[0, 1, 2, 3, 4, 5, 6, 7, 8]